# Lab03 - SAT solving with Z3 in Python

We will be using Z3 programmatically in Python. Consider the following classical logic tautologies:

$$

\varphi_1 \equiv\ p \vee \neg p \qquad \textrm{(excluded middle)} \\
\varphi_2 \equiv\ ((p \to q) \to p) \to p \qquad \textrm{(Pierce's law)}
$$
The Z3 code below (via its convenient Python interface) checks that the formulas above are tautologies by trying to find a satisfying assignment to their negation.

In [1]:
from z3 import *

# create a fresh propositional variable uniquely identified by its name 'p'
p = Bool('p')

# excluded middle
solve(Not(Or(p, Not(p))))

# Pierce's law
q = Bool('q')
solve(Not(Implies(Implies(Implies(p, q), p), p)))

#Other useful connectives:
#Implies, And

no solution
no solution


# Exercise (tautologies)

Prove that the following propositional formula is a classical tautology by checking that its negation is unsatisfiable: $((p \land q) \to r) \leftrightarrow (p \to (q \to r))$.

What about the following Łukasiewicz's formula? $((p \to q) \to r) \to ((r \to p) \to (s \to p))$.

In [2]:
r = Bool('r')
s = Bool('s')

solve(Not(Implies(And(p, q), r) == Implies(p, Implies(q, r))))
solve(Not(Implies(Implies(Implies(p, q), r), Implies(Implies(r, p), Implies(s, p)))))

no solution
no solution


# Example (long conjunctions and disjunctions)

We can conveniently use lists to represent long conjunctions `And` and disjunctions `Or`. For instance the next cell implements the following formula: $x_1 \wedge x_2 \wedge \cdots \wedge x_n$.

In [3]:
n = 10
x = [Bool('x' + str(k)) for k in range(n)]
phi = And([x[k] for k in range(n)])
print(phi)

solve(phi) # satisfiable

And(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9)
[x2 = True,
 x1 = True,
 x0 = True,
 x3 = True,
 x7 = True,
 x9 = True,
 x4 = True,
 x5 = True,
 x6 = True,
 x8 = True]


# Exercise (All satisfying assignments)

If Z3 solves a formula, it returns some satisfying assignment. How can we use Z3 in order to construct all satisfying assignments? Tip:o make this meaningful, if the number of satisfying assignments is $n$, then we want to call Z3 $n$ times. Write a Python program that returns all satisfying assignments of a given SAT instance.

In [4]:
from z3 import *

# this procedure returns an assignment (as a dictionary) if any exists
def model(phi):
    # create a SAT instance
    s = Solver()
    s.add(phi)
    # return a satisfying assignment
    return s.model() if s.check() == sat else {}


def all_models(phi):
  models = []
  while True:
    m = model(phi)
    if not m:
      break
    models.append(m)
    block = []
    for d in m:
      c = d()
      block.append(c != m[d])
    phi = And(phi, Or(block))
  if not models:
    print("no solution")
  return models


p = Bool("p")
q = Bool("q")
r = Bool("r")
phi = Or(And(p, Not(q)), r)
m = model(phi)
print(p, '=', m[p])
print(q, '=', m[q])
print(r, '=', m[r])

# Pro tip 1: one can enumerate all variables by looking at the keys of m
# Pro tip 2: if x is a key of m,
# the corresponding Z3 variable can be reconstructed with "Bool(str(x))"
# (it would be cool if it was just "x", but it is not :)
vars = [Bool(str(x)) for x in m]

ms = all_models(phi)
for m in ms:
  print()
  print(p, '=', m[p])
  print(q, '=', m[q])
  print(r, '=', m[r])

p = True
q = False
r = False

p = True
q = False
r = False

p = False
q = False
r = True

p = True
q = False
r = True

p = True
q = True
r = True

p = False
q = True
r = True


# Dimacs format

The DIMACS format is a concrete syntax for propositional formulas in CNF. The following example of the DIMACS syntax corresponds to the formula:

$(𝑥_1∨¬𝑥_3) ∧ (𝑥_2∨𝑥_3∨¬𝑥_1)$

```no-exec id=81816403-88ad-442a-b1dc-8d571a5465d6
c this is a comment and the next line says that this SAT instance has 3 variables and 2 clauses
p cnf 3 2
1 -3 0
2 3 -1 0
```

The comment line starts with c. The next line states that the formula is in the cnf format and there are 3 variables and 2 clauses.

The first clause contains "x₁" and "not x₃", "0" is a terminator that allows clauses to span multiple lines. The second clause contains "x₂", "x₃", and "not x₁"

Such a file can be directly given to the Z3 standalone prover (using the "z3 file" command).

In [5]:
%%script z3 -dimacs -in

p cnf 3 2
1 -3 0
2 3 -1 0

s SATISFIABLE
v -1 -2 -3 0


The output is a satisfying assignment, in this case all three variables are set to false.

# Exercise (**Pebbling formulas**)

This exercise shows that solvers can be fast on certain (unsatisfiable) fragments of SAT. For an $𝑛\times 𝑛$ grid, the $p_n$ *pebbling formula* says that: 

1. There is a pebble on the top left vertex.
2. A vertex is pebbled if both the neighbour to the left and above are pebbled.
3. There is no pebble on the bottom right vertex.

The formula $p_n$ is clearly unsatisfiable. We are interested to determine whether this is a hard or an easy SAT instance. 

* Write down the CNF formula *$p_𝑛$.* Does this formula belong to some efficient class of SAT instances? 
* Write a Python program using Z3 to test satisfiability for $p_𝑛$*.*  For how large *$n$ *is the running time feasible (e.g., <1 sec.)?
* Write the formula $p_n$ in DIMACS format and check how fast is Z3 without Python interface (run Z3 from the command line).  For how large *$n$ *is the running time feasible (e.g., <1 sec.)?

In [6]:
from z3 import *
from datetime import datetime

# should a dimacs output be produced (on stdout) or rather
# a python z3 formula (and the solver called)?
dimacs = True

# grid size: n×n
n = 800

#(0,0)
#  1 → ? → …
#  ↓   ↓
#  ? → ? → …
#  ↓   ↓
#  …   …
#            0 
#        (n-1,n-1)

if not dimacs:
    # Create a n×n "matrix" (list of lists) of boolean variables
    X = [[ Bool("x_%s_%s" % (i, j)) for j in range(n) ]
         for i in range(n)]
    
    s = Solver()
    
    # 2. A vertex is pebbled if both the neighbours to the left and above are pebbled
    for i in range(n):
        for j in range(n):
            if i == 0 or j == 0:
                # 1. There is a pebble on the top left vertex
                s.add(X[i][j])
            else:
                pebbled = And(X[i][j], X[i-1][j], X[i][j-1])
                no_pebbled = And(Not(X[i][j]), Or(Not(X[i-1][j]), Not(X[i][j-1])))
                s.add(Or(pebbled, no_pebbled))
    
    # 3. There is no pebble on the bottom right vertex
    s.add(Not(X[n-1][n-1]))
    
    start = datetime.now()
    res = s.check()
    elapsed = datetime.now() - start
    
    print("Satisfiable: ", res == sat)
    print("Elapsed time: ", elapsed)
else:
  with open("pebbling_formula.dimacs", "w") as f:
    f.write(f"p cnf {n*n} {3*n*n-4*n+3}\n")
    
    # 2. A vertex is pebbled if both the neighbours to the left and above are pebbled
    for i in range(n):
        for j in range(n):
            if i == 0 or j == 0:
                # 1. There is a pebble on the top left vertex
                f.write(f"{i*n + j + 1} 0\n")
            else:
                f.write(f"{i*n + j + 1} {(i-1)*n + j + 1} {i*n + j} 0\n")
                f.write(f"-{i*n + j + 1} -{(i-1)*n + j + 1} 0\n")
                f.write(f"-{i*n + j + 1} -{i*n + j} 0\n")
    
    # 3. There is no pebble on the bottom right vertex
    f.write(f"-{n*n} 0\n")

In [7]:
%%sh
z3 -dimacs pebbling_formula.dimacs

s UNSATISFIABLE


Czas działania wynosi $\sim 1s$ używając Z3 dla $n=300$, natomiast dla $n=800$ używając DIMACSa.

# Exercise (Queens)

The eight queens puzzle is the problem of placing eight [chess](https://en.wikipedia.org/wiki/Chess) [queens](https://en.wikipedia.org/wiki/Queen_(chess)) on an 8×8 [chessboard](https://en.wikipedia.org/wiki/Chessboard) so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. The eight queens puzzle is a special case of the more general n queens problem of placing *n* non-attacking queens on an *n*×*n* chessboard. Solutions exist for all [natural numbers](https://en.wikipedia.org/wiki/Natural_number) *n* with the exception of *n* = 2 and *n* = 3. 

Write a program that computes a solution for given n.

In [8]:
from z3 import *

# chessboard size
n = 4

# Create a rxr "matrix" (list of lists) of boolean variables
X = [ [ Bool("x_%s_%s" % (i, j)) for j in range(n) ]
      for i in range(n) ]
print(X)

# this procedure returns an assignment (as a dictionary) if any exists
def model(phi):
    # create a SAT instance
    s = Solver()
    s.add(phi)

    # return a satisfying assignment
    return s.model() if s.check() == sat else {}


# this procedure draws the board corresponding to the model, e.g:
# - - H -
# H - - -
# - - - H
# - H - -
def drawBoard(model):
    print("Verdict for n =", n)
    if not model:
        print("No solution")
    else:
        print("Solution found:")
        for i in range(n):
            for j in range(n):
                print("H" if model[X[i][j]] else "-", end=" ")
            print()

# pro tips:
# 1) there must be a queen in every row
# 2) no two queens on the same column
#    (i.e. for any column k and any two rows i, j, the queen is not in (i,k) or not in (j,k))
# (the two above imply that there must be exactly one queen in each row and each column)
# 3) no two queens on the same "positive" diagonal
# 4) no two queens on the same "negative" diagonal

phi_1 = [Or([X[i][j] for j in range(n)]) for i in range(n)]

phi_2 = [Or(Not(X[i][k]), Not(X[j][k])) for k in range(n) for i in range(n) for j in range(i + 1, n)]

phi_3 = [Or(Not(X[i][j]), Not(X[k][l]))
         for i in range(n) for j in range(n) for k in range(n) for l in range(n)
         if i < k and i + j == k + l]

phi_4 = [Or(Not(X[i][j]), Not(X[k][l]))
         for i in range(n) for j in range(n) for k in range(n) for l in range(n)
         if i < k and i - j == k - l]

phi = phi_1 + phi_2 + phi_3 + phi_4
solution = model(phi)
drawBoard(solution)

[[x_0_0, x_0_1, x_0_2, x_0_3], [x_1_0, x_1_1, x_1_2, x_1_3], [x_2_0, x_2_1, x_2_2, x_2_3], [x_3_0, x_3_1, x_3_2, x_3_3]]
Verdict for n = 4
Solution found:
- - H - 
H - - - 
- - - H 
- H - - 
